# Asset e passività

All'inizio del 2000 abbiamo avuto una crisi dei fondi pensione. Da dicembre 1999 a maggio 2003 siamo passati da una attivo di 239 miliardi a un passivo di 252 miliardi con una spaventosa perdita di circa 500 miliardi.

Cosa è successo all'inizio del 2000 abbiamo avuto una bolla sul settore tecnologico con una forte contrazione sul mercato azionario, questo ha fatto ridurre gli attivi dei fondi pensionistici che investivano almeno una parte in azionariato.

Ora se fosse stato solo per il mercato azionario questa cosa sarebbe stata brutta, ma la situazione è passata da brutta a spaventosa.
Il taglio dei tassi di interesse ha fatto abbassare anche i rendimenti delle obbligazioni con il conseguente aumento dei debiti di questi fondi.

La passività di un fondo pensione è simile ad un fondo obbligazionario (il fondo pensione si impegna a restituire il capitale investito più una certa quota di interessi fissi). dunque il taglio dei tassi di interesse ha fatto si che che la passività di questi fondi aumentasse, questa è stata una grave falla nella gestione del rischio. Stessa cosa è accaduta nel 2008.

Concludendo nella gestione patrimoniale ciò che conta è la gestione dell'attivo $A_t$ rispetto al debito $P_t$.

$$
F_t = \frac{A_t}{P_t}
$$

Definiamo come surplus o deficit

$$
S_t = A_t - P_t
$$

Se il valore $F_t$ è maggiore di 100 saremo sicuramente capaci di tener testa ai nostri impegni.

## Valore attuale della passività

Cominciamo con l'esaminare il valore delle passività future per farlo dobbiamo attualizzare il valore futuro, chiameremo questo valore **Present Value** facciamo un esempio pratico.

Supponiamo di dover pagare fra 5 anni $n$ una somma di 1000€ $FV$, diamo come tasso di interesse il 3% $r$ quanto dovrei avere a disposizione ora?

$$
PV = \frac{FV}{(1+r)^n}
$$

Che a conti fatti danno un present value di circa 863€
Ovviamente i tassi di interessi cambiano nel tempo perciò il calcolo della nostra passività sarà di 

$$
PV(L) = \sum_{i=1}^{k} B(t_i) L_i
$$

Il valore $B(t_i)$ rappresenta il valore delle obbligazioni scontate che valgono un euro al tempo $t_i$, se assumiamo che la curva dei rendimenti è piatta e il tasso di interesse è $r$ avremo che

$$
B(t_i) = \frac{1}{(1+r)^t}
$$

iniziamo con l'importazione delle librerie e la scrittura della funzione B

In [1]:
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2

In [2]:
def discount(t, r):
    """
    Compute the price of a pure discount bond that pays $1 at time t where t is in years and r is the annual interest rate
    """
    return (1+r)**(-t)

Calcoliamo ora il prezzo scontato di un euro di una obbligazione a 10 anni con un tasso di interesse al 3%

In [3]:
b = discount(10, .03)
b

0.7440939148967249

Verifichiamo che il codice sia corretto

In [4]:
b*(1.03**10)

1.0

Abbiamo quello che ci serve possiamo scrivere anche la funzione $PV$

In [5]:
def pv(l, r):
    """
    Compute the present value of a list of liabilities given by the time (as an index) and amounts
    """
    dates = l.index
    discounts = discount(dates, r)
    return (discounts*l).sum()

Assumiamo ora di avere 4 debiti di 1,1.5,2 e 2.5 Milioni di euro. Assumiamo che il primo sia a scadenza di 3 anni e gli altri siano a scadenze di 6 mesi a incrementare. Calcoliamo il valore attualizzato dei debiti.

In [6]:
liabilities = pd.Series(data=[1, 1.5, 2, 2.5], index=[3, 3.5, 4, 4.5])

In [7]:
pv(liabilities, 0.03)

6.233320315080045

Ora assumiamo di calcolare il rapporto di copertura dei finanziamenti **Founding ratio** basandosi sul valore corrente degli attivi:

In [8]:
def funding_ratio(assets, liabilities, r):
    """
    Computes the funding ratio of a series of liabilities, based on an interest rate and current value of assets
    """
    return assets/pv(liabilities, r)

In [9]:
funding_ratio(5, liabilities, 0.03)

0.8021407126958777

abbassiamo il tasso di interessa dal 3% al 2%

In [10]:
funding_ratio(5, liabilities, 0.02)

0.7720304366941648

In [11]:
import ipywidgets as widgets
from IPython.display import display
%matplotlib inline

def show_funding_ratio(assets, r):
    fr = funding_ratio(assets, liabilities, r)
    print(f'{fr*100:.2f}%')
    
controls = widgets.interactive(show_funding_ratio,
                                   assets=widgets.IntSlider(min=1, max=10, step=1, value=5),
                                   r=(0, .20, .01)
)
display(controls)

interactive(children=(IntSlider(value=5, description='assets', max=10, min=1), FloatSlider(value=0.1, descript…